In [1]:
import pandas as pd
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, get_cosine_schedule_with_warmup
from transformers.utils import is_flash_attn_2_available
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from accelerate import Accelerator


In [2]:
!python --version
!which python

Python 3.12.9
/home/bs_ms/sbs22ms076/fire/env/bin/python


In [3]:
quantization_config= BitsAndBytesConfig(load_in_8bit=True)
use_quantization_config= True

attention_implementation= 'eager'
model_id='google/gemma-3-4b-it'
tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id,padding_side="left")
llm_model=AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                        torch_dtype=torch.bfloat16,
                                        quantization_config=quantization_config if use_quantization_config else None
                                        )#attn_implementation=attention_implementation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# # Instead of trying to attach a head to a non-existent attribute,
# # we will create it as a separate module.
# # The input to this head will be the output of the Gemma model.
# # Gemma 3B's hidden size is 2560.
# hidden_size = llm_model.config.hidden_size
# print(f"Model hidden size: {hidden_size}")

# classification_head = nn.Sequential(
#     nn.LayerNorm(hidden_size),
#     nn.Dropout(0.4),
#     nn.Linear(in_features=hidden_size, out_features=1024, dtype=torch.float32),
#     nn.GELU(),
#     nn.Dropout(0.4),
#     nn.Linear(in_features=1024, out_features=512, dtype=torch.float32),
#     nn.GELU(),
#     nn.Dropout(0.4),
#     nn.Linear(in_features=512, out_features=8, dtype=torch.float32) # 8 classes
# ).to(device)

# # Freeze the original LLM parameters
# for param in llm_model.parameters():
#     param.requires_grad = False

AttributeError: 'Gemma3Config' object has no attribute 'hidden_size'

In [4]:

encoded_text=['kya baaat','sab changa si','moj kardi bete waah bete waah                               ']
max((len(encoding) for encoding in encoded_text),default=0)

60

In [5]:
tokenizer.pad_token = tokenizer.eos_token
class Spam_dataset(Dataset):
    def __init__(self, csv_file, tokenizer, Max_length=None):
        self.data=pd.read_csv(csv_file)
        self.encoded_text=[tokenizer.encode(text) for text in self.data['Text']]
        texts = list(self.data['Text'])

        if Max_length == None:
            self.max_length=self._longest_length()
        else:
            self.max_length=Max_length

        self.tokenized = tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        self.labels = torch.tensor(self.data['Datatype'].values, dtype=torch.long)
    def _longest_length(self):
        return max((len(encoding) for encoding in self.encoded_text),default=0)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label=self.data.iloc[index]['Datatype']
        return {
            'input_ids': self.tokenized['input_ids'][index],
            'attention_mask': self.tokenized['attention_mask'][index]
        }, torch.tensor(label,dtype=torch.long)


In [6]:
tokenizer.pad_token_id

1

In [7]:
train_dataset=Spam_dataset(csv_file='youtube_train.csv',tokenizer=tokenizer,Max_length=None)
test_dataset=Spam_dataset(csv_file='youtube_test.csv',tokenizer=tokenizer,Max_length=train_dataset.max_length)
vali_dataset=Spam_dataset(csv_file='youtube_validation.csv',tokenizer=tokenizer,Max_length=train_dataset.max_length)

In [8]:
torch.manual_seed(693)
worker_size=0
batch_size=4

train_loader= DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         num_workers=worker_size,
                         shuffle=True,
                         drop_last=True
                         )
vali_loader= DataLoader(dataset=vali_dataset,
                         batch_size=batch_size,
                         num_workers=worker_size,
                         shuffle=True,
                         drop_last=True
                         )
test_loader= DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         num_workers=worker_size,
                         shuffle=True,
                         drop_last=True
                         )

for 1 billion model it's syntax is llm_model.lm_head.weight.shape

In [9]:
for param in llm_model.parameters():
    param.requires_grad=False

# llm_model.out_head=nn.Sequential(nn.LayerNorm(2560),nn.Dropout(p=0.2),nn.Linear(in_features=2560 ,out_features=8,dtype=torch.float32)).to(device)
llm_model.language_model.out_head=nn.Sequential(nn.LayerNorm(2560),nn.Dropout(0.4),nn.Linear(in_features=2560 ,out_features=1024,dtype=torch.float32),nn.GELU()
                                 ,nn.Dropout(0.4),nn.Linear(in_features=1024 ,out_features=512,dtype=torch.float32),nn.GELU(),
                                  nn.Dropout(0.4),nn.Linear(in_features=512 ,out_features=8,dtype=torch.float32)).to(device)
# print("head weight dtype:", llm_model.out_head.weight.dtype)


In [10]:
for name,param in llm_model.language_model.model.layers[-1].named_parameters():
    if param.is_floating_point():
        print(f"available for grad {name} and it's type {param.dtype}")
        param.requires_grad=True
        
    else:
        print(f'skipping-:{name}  type{param.dtype} ')

for param in llm_model.language_model.model.norm.parameters():
    param.requires_grad = True
for param in llm_model.language_model.lm_head.parameters():
    param.requires_grad = True
for param in llm_model.language_model.out_head.parameters(): #------>tried training my out head when arcihtecture was layer norm(1156),dropout,(1156,576),gelu,dropout,(576,8)
    param.requires_grad = True

skipping-:self_attn.q_proj.weight  typetorch.int8 
skipping-:self_attn.k_proj.weight  typetorch.int8 
skipping-:self_attn.v_proj.weight  typetorch.int8 
skipping-:self_attn.o_proj.weight  typetorch.int8 
available for grad self_attn.q_norm.weight and it's type torch.bfloat16
available for grad self_attn.k_norm.weight and it's type torch.bfloat16
skipping-:mlp.gate_proj.weight  typetorch.int8 
skipping-:mlp.up_proj.weight  typetorch.int8 
skipping-:mlp.down_proj.weight  typetorch.int8 
available for grad input_layernorm.weight and it's type torch.bfloat16
available for grad post_attention_layernorm.weight and it's type torch.bfloat16
available for grad pre_feedforward_layernorm.weight and it's type torch.bfloat16
available for grad post_feedforward_layernorm.weight and it's type torch.bfloat16


In [11]:
llm_model

Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear8bitLt(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear8bitLt(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear8bitLt(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear8bitLt(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp):

In [12]:

text= 'waht are protein shake brother ??'
input_id= tokenizer(text,return_tensors='pt')

# embed_device = llm_model.language_model.model.get_input_embeddings().weight.device
input_id = {k: v.to(device) for k, v in input_id.items()}
print(input_id)

with torch.no_grad():
    outputs = llm_model.language_model.model(**input_id)  
    last_token = outputs.last_hidden_state[:, -1, :]  
    
print("Last token max:", last_token.max().item(), "min:", last_token.min().item())
logits = llm_model.language_model.out_head(last_token.float())
print("Logits:", logits)
print("Logits shape:", logits.shape)
norm=torch.softmax(logits,dim=-1)        #you can keep dim=-1 for same output 
maxi=torch.argmax(logits,dim=-1) #you can keep dim=1 same output 
print(maxi)
print(norm)


{'input_ids': tensor([[    2,  3886,   613,   659,  6762, 31716, 10070, 18885]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


/home/bs_ms/.local/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Last token max: 30.875 min: -18.0
Logits: tensor([[ 0.1144, -0.0674,  0.0183,  0.0104,  0.2213, -0.2454, -0.1271, -0.1769]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
Logits shape: torch.Size([1, 8])
tensor([4], device='cuda:0')
tensor([[0.1431, 0.1193, 0.1300, 0.1290, 0.1593, 0.0999, 0.1124, 0.1070]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [13]:
def calc_accuracy_loader( data_loader, model, device,num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch = {k: v.to(device) for k, v in input_batch.items()}
            target_batch = target_batch.to(device)

            with torch.no_grad():
                # print("input_ids:", input_batch["input_ids"])
                outputs = llm_model.language_model.model(**input_batch)
                last_token = outputs.last_hidden_state[:, -1, :]
                # print("last_hidden_state",last_token)
                logits = llm_model.language_model.out_head(last_token.float())
                # print('logits',logits)  
            predicted_labels=torch.argmax(logits,dim=1)
            # print('predicted_labels',predicted_labels)
            num_examples += logits.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break
    return correct_predictions / num_examples

In [14]:

train_accuracy = calc_accuracy_loader(train_loader, model=llm_model, device=device, num_batches=8)
train_accuracy

0.34375

In [15]:
# label_penalty=[400,230,70,16,40,30,10,220]
label_penalty=[715,448,178,48,122,87,30,428]
label_tens=torch.tensor(label_penalty,dtype=torch.float)
weights=1/label_tens
weights=(weights/weights.sum())*len(label_penalty)
weights.to(device)

tensor([0.1310, 0.2090, 0.5260, 1.9506, 0.7675, 1.0762, 3.1210, 0.2188],
       device='cuda:0')

In [16]:
def loss_func(input_batch,target_batch,model,device,weights=weights):
    input_batch = {k: v.to(device) for k, v in input_batch.items()}
    target_batch = target_batch.to(device)
    # print("target shape:", target_batch.shape)
    # print("target min:", target_batch.min().item(), "max:", target_batch.max().item())
    outputs = model.language_model.model(**input_batch)
    last_token = outputs.last_hidden_state[:, -1, :]
    # print("Last token max:", last_token.max().item(), "min:", last_token.min().item())
    if torch.isnan(last_token).any():
        print("⚠️ last_token has NaNs!")
    if torch.isinf(last_token).any():
        print("⚠️ last_token has Infs!")
    logits = model.language_model.out_head(last_token.float())
    # print("logits shape:", logits.shape)
    if torch.isnan(logits).any():
        print("⚠️ logits contain NaN!")
    if torch.isinf(logits).any():
        print("⚠️ logits contain Inf!")
    weights=weights.to(device=device)
    
    loss = torch.nn.functional.cross_entropy(logits, target_batch,weight=weights)
    return loss

In [17]:
def calc_loss_accuracy(dataloader,device,model,num_batches=None):
    total_loss=0
    if num_batches == None:
        num_batches=len(dataloader)
    else:
        num_batches= min(num_batches,len(dataloader))
    for i,(input_batch,target_batch) in enumerate(dataloader):
        if i<num_batches:
            # print("target min:", target_batch.min().item(), "max:", target_batch.max().item())
            loss= loss_func(input_batch=input_batch,target_batch=target_batch,model=model,device=device)
            total_loss+= loss.item()
        else:
            break
    return total_loss/num_batches

    

In [18]:
def evaluate_model(model, train_loader, vali_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_accuracy(dataloader=train_loader,device=device,model=model, num_batches=eval_iter)
        vali_loss = calc_loss_accuracy(dataloader=vali_loader,device=device,model=model, num_batches=eval_iter)
    model.train()
    return train_loss, vali_loss

In [19]:

def train_classifier_simple(model, train_loader, vali_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter,scheduler,best_val_accuracy=0.75):
    
    train_losses, vali_losses, train_accs, vali_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    
    for epoch in range(num_epochs):
        model.train()  

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() 
            loss = loss_func(input_batch=input_batch,target_batch=target_batch,model=model,device=device)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            loss.backward() 
            optimizer.step()
            scheduler.step()
            examples_seen += input_batch['input_ids'].shape[0]
            global_step += 1

           
            if global_step % eval_freq == 0:
                torch.cuda.empty_cache()

                train_loss, vali_loss = evaluate_model(
                    model=model,train_loader= train_loader, vali_loader=vali_loader, device=device, eval_iter=eval_iter)
                train_losses.append(train_loss)
                vali_losses.append(vali_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {vali_loss:.3f}")
                
                torch.cuda.empty_cache()
                train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
                vali_accuracy = calc_accuracy_loader(vali_loader, model, device, num_batches=eval_iter)
                print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
                print(f"Validation accuracy: {vali_accuracy*100:.2f}%")
                train_accs.append(train_accuracy)
                vali_accs.append(vali_accuracy)

                if vali_accuracy >= best_val_accuracy:
                    if (train_accuracy - vali_accuracy) <=0.05:
                        best_val_accuracy = vali_accuracy 
                        torch.save({
                            'model_state_dict': llm_model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'scheduler_state_dict': scheduler.state_dict(),
                        }, f'youtube_classifier_model{int(train_accuracy*100)}_{int(vali_accuracy*100)}_.pt')
                        print(f"Model saved with improved accuracy: {vali_accuracy:.4f}")

        torch.cuda.empty_cache()

    return train_losses, vali_losses, train_accs, vali_accs, examples_seen

In [20]:
import time

start_time = time.time()

torch.manual_seed(693)

optimizer = torch.optim.AdamW(llm_model.parameters(), lr=5e-5, weight_decay=0.1)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
warmup_steps = int(0.15 * total_steps)  # 10% warmup
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

train_losses, vali_losses, train_accs, vali_accs, examples_seen = train_classifier_simple(
    model=llm_model, train_loader=train_loader, vali_loader=vali_loader, optimizer=optimizer, device=device,
    num_epochs=num_epochs, eval_freq=40, eval_iter=8,scheduler=scheduler
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 70.00 MiB. GPU 0 has a total capacity of 23.46 GiB of which 62.06 MiB is free. Including non-PyTorch memory, this process has 23.37 GiB memory in use. Of the allocated memory 22.43 GiB is allocated by PyTorch, and 759.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print([name for name, _ in llm_model.named_parameters() if 'out_head' in name])

In [1]:
# train_spam.py

import pandas as pd
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, get_cosine_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from accelerate import Accelerator # You already have this!
import time

# ==========================================================
# 1. Initialize Accelerator at the beginning of your script
# ==========================================================
accelerator = Accelerator()

# The device will be handled by Accelerator
device = accelerator.device

# --- Model and Tokenizer Loading (No changes here, but good) ---
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
use_quantization_config = True
model_id = 'google/gemma-3-4b-it'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

# Load model, but DO NOT move to device with .to(device) yet
llm_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config if use_quantization_config else None,
    # For better performance on your RTX cards, enable flash attention
    attn_implementation="eager" 
)
print(llm_model.config)
# --- Your Custom Head ---
# NEW, correct line:
# NEW, correct line for your multimodal model:
hidden_size = llm_model.config.text_config.hidden_size
# Add this line to your code to see all available config options

classification_head = nn.Sequential(
    nn.LayerNorm(hidden_size),
    nn.Dropout(0.4),
    nn.Linear(in_features=hidden_size, out_features=1024, dtype=torch.float32),
    nn.GELU(),
    nn.Dropout(0.4),
    nn.Linear(in_features=1024, out_features=512, dtype=torch.float32),
    nn.GELU(),
    nn.Dropout(0.4),
    nn.Linear(in_features=512, out_features=8, dtype=torch.float32)
)

# Freeze LLM
for param in llm_model.parameters():
    param.requires_grad = False

# --- Dataset and Dataloader (No changes to the class itself) ---
class Spam_dataset(Dataset):
    # ... your class code is fine ...
    def __init__(self, csv_file, tokenizer, Max_length=None):
        self.data=pd.read_csv(csv_file)
        texts = list(self.data['Text'])

        if Max_length == None:
            encoded_text=[tokenizer.encode(text) for text in self.data['Text']]
            self.max_length = max((len(encoding) for encoding in encoded_text),default=0)
        else:
            self.max_length=Max_length

        self.tokenized = tokenizer(
            texts, padding='max_length', truncation=True,
            max_length=self.max_length, return_tensors='pt'
        )
        self.labels = torch.tensor(self.data['Datatype'].values, dtype=torch.long)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # The label is now part of the dictionary for easier handling with accelerate
        return {
            'input_ids': self.tokenized['input_ids'][index],
            'attention_mask': self.tokenized['attention_mask'][index],
            'labels': self.labels[index]
        }

train_dataset = Spam_dataset(csv_file='youtube_train.csv', tokenizer=tokenizer, Max_length=None)
vali_dataset = Spam_dataset(csv_file='youtube_validation.csv', tokenizer=tokenizer, Max_length=train_dataset.max_length)

train_loader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
vali_loader = DataLoader(dataset=vali_dataset, batch_size=4, shuffle=False)

# --- Optimizer and Scheduler ---
# We combine parameters from the head. Only these will be updated.
optimizer = torch.optim.AdamW(classification_head.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 3
total_steps = len(train_loader) * num_epochs
warmup_steps = int(0.15 * total_steps)
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

# ==========================================================
# 2. Use accelerator.prepare()
# This is the magic step. It wraps your models, optimizer, and dataloaders.
# ==========================================================
llm_model, classification_head, optimizer, train_loader, vali_loader, scheduler = accelerator.prepare(
    llm_model, classification_head, optimizer, train_loader, vali_loader, scheduler
)

# --- Loss Weights (Don't move to device manually) ---
label_penalty = [715, 448, 178, 48, 122, 87, 30, 428]
label_tens = torch.tensor(label_penalty, dtype=torch.float)
weights = 1 / label_tens
weights = (weights / weights.sum()) * len(label_penalty)
# Move weights to the correct device inside the training loop

# (Assuming your accelerator.prepare() call is done before this)

# --- Define logging frequency ---
log_interval = 50  # Print training progress every 50 steps

# --- Training Loop ---
best_val_accuracy = 0.0
accelerator.print("=======================================")
accelerator.print("           STARTING TRAINING           ")
accelerator.print("=======================================")

for epoch in range(num_epochs):
    accelerator.print(f"\n--- Starting Epoch {epoch+1}/{num_epochs} ---")
    
    # --- Set models to correct mode ---
    llm_model.eval()  # Base model is always in eval mode as it's frozen
    classification_head.train()
    
    # --- Training Phase for the current epoch ---
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        with torch.no_grad():
            outputs = llm_model(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                output_hidden_states=True
            )
            last_hidden_state = outputs.hidden_states[-1]
            last_token_hidden_state = last_hidden_state[:, -1, :]
            
        logits = classification_head(last_token_hidden_state.float())
        loss = torch.nn.functional.cross_entropy(logits, labels, weight=weights.to(device))
        
        # Print progress periodically
        if step % log_interval == 0 and step > 0:
            accelerator.print(f"  [Epoch {epoch+1}, Step {step}/{len(train_loader)}] Training batch loss: {loss.item():.4f}")

        accelerator.backward(loss)
        optimizer.step()
        scheduler.step()
        
    accelerator.print(f"--- Finished training for Epoch {epoch+1} ---")

    # --- Evaluation Phase for the current epoch ---
    accelerator.print(f"\n--- Starting evaluation for Epoch {epoch+1} ---")
    llm_model.eval()
    classification_head.eval()
    total_eval_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for i, batch in enumerate(vali_loader):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = llm_model(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                output_hidden_states=True
            )
            last_hidden_state = outputs.hidden_states[-1]
            last_token_hidden_state = last_hidden_state[:, -1, :]

            logits = classification_head(last_token_hidden_state.float())
            loss = torch.nn.functional.cross_entropy(logits, labels, weight=weights.to(device))
            
            total_eval_loss += loss.item()
            
            preds = torch.argmax(logits, dim=-1)
            gathered_preds = accelerator.gather(preds)
            gathered_labels = accelerator.gather(labels)
            
            correct_predictions += (gathered_preds == gathered_labels).sum().item()
            total_samples += gathered_labels.size(0)

    avg_val_loss = total_eval_loss / len(vali_loader)
    val_accuracy = correct_predictions / total_samples

    # Print the final evaluation results for the epoch
    accelerator.print(f"--- Evaluation complete for Epoch {epoch+1} ---")
    accelerator.print(f"  => Epoch {epoch+1} Results: Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Check for improvement and save the model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        accelerator.print(f"  ✨ New best accuracy: {best_val_accuracy:.4f}. Saving model... ✨")

        accelerator.wait_for_everyone()
        unwrapped_head = accelerator.unwrap_model(classification_head)
        accelerator.save(unwrapped_head.state_dict(), f"youtube_classifier_head_acc_{val_accuracy:.2f}.pt")
        accelerator.print("  ✅ Model saved.")

accelerator.print("\n=======================================")
accelerator.print("           TRAINING COMPLETE           ")
accelerator.print("=======================================")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma3Config {
  "_attn_implementation_autoset": true,
  "architectures": [
    "Gemma3ForConditionalGeneration"
  ],
  "boi_token_index": 255999,
  "eoi_token_index": 256000,
  "eos_token_id": [
    1,
    106
  ],
  "image_token_index": 262144,
  "initializer_range": 0.02,
  "mm_tokens_per_image": 256,
  "model_type": "gemma3",
  "quantization_config": {
    "_load_in_4bit": false,
    "_load_in_8bit": true,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "attn_logit_softcapping": null,
    "cache_implementation": "hybrid",
    "final_logit_softcapping": null

/home/bs_ms/.local/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


  [Epoch 1, Step 50/875] Training batch loss: 1.9114
  [Epoch 1, Step 100/875] Training batch loss: 1.9048
  [Epoch 1, Step 150/875] Training batch loss: 1.6292
  [Epoch 1, Step 200/875] Training batch loss: 1.9586
  [Epoch 1, Step 250/875] Training batch loss: 1.2660
  [Epoch 1, Step 300/875] Training batch loss: 0.9970
  [Epoch 1, Step 350/875] Training batch loss: 1.1883
  [Epoch 1, Step 400/875] Training batch loss: 1.1585
  [Epoch 1, Step 450/875] Training batch loss: 1.2181
  [Epoch 1, Step 500/875] Training batch loss: 1.1943
  [Epoch 1, Step 550/875] Training batch loss: 0.6756
  [Epoch 1, Step 600/875] Training batch loss: 0.7232
  [Epoch 1, Step 650/875] Training batch loss: 0.7120
  [Epoch 1, Step 700/875] Training batch loss: 0.7846
  [Epoch 1, Step 750/875] Training batch loss: 0.7457
  [Epoch 1, Step 800/875] Training batch loss: 0.7083
  [Epoch 1, Step 850/875] Training batch loss: 1.0695
--- Finished training for Epoch 1 ---

--- Starting evaluation for Epoch 1 ---
--- 